In [1]:
import pandas as pd

# Cargar datos
df = pd.read_csv("/content/Base de datos FINAL 2 .csv", sep=";")


# Echar un vistazo
df.head(5)

,nombre-pelicula,anio,tipo-largometraje,duracion-min,tipo-filme,color,director-01,idioma,consol-idioma,lugar-prod,consol-lugar-prod,pais-rodaje,consol-pais-rodaje,,lugar-estreno,pais-estreno,sinopsis
0,La industria del salitre,1915,Largometraje documental,NI,35 mm,"Blanco & negro, Muda",Francisco Caamaño,Español,Español,Chile,Chile,Chile,Chile,"Teatros Setiembre y Politeama,Chile",Teatros Setiembre y Politeama,Chile,Largometraje sobre el proceso de extracción y ...
1,La industria del salitre,1915,Largometraje documental,NI,35 mm,"Blanco & negro, Muda",Francisco Caamaño,Español,Español,Chile,Chile,Chile,Chile,"Teatros Setiembre y Politeama,Chile",Teatros Setiembre y Politeama,Chile,Largometraje sobre el proceso de extracción y ...
2,La baraja de la muerte (o El enigma de la call...,1916,Largometraje de ficción,100,35 mm,"Blanco & negro, Muda",Salvador Giambastiani,Español,Español,Chile,Chile,Chile,Chile,"Teatros Colón y Novedades,Chile",Teatros Colón y Novedades,Chile,Basa su acción en un crimen cometido por Corin...
3,El hombre de acero,1917,Largometraje de ficción,NI,35 mm,"Blanco & negro, Muda",Jorge Délano «Coke»,Español,Español,Chile,Chile,Chile,Chile,Teatros Unión Central y Septiembre,NI,NI,Creación Colectiva bajo la dirección de Pedro ...
4,La agonía de Arauco (o El olvido de los muertos),1917,Largometraje de ficción,NI,35 mm,"Blanco & negro, Muda",Gabriela Bussenius,Español,Español,Chile,Chile,Chile,Chile,Teatros Unión Central y Alhambra,NI,NI,"Es una aventura sentimental, donde el paisaje ..."


Para visualizar ubicaciones en un mapa, necesitamos coordenadas geográficas (latitud y longitud). La columna `pais-estreno` contiene nombres de países, por lo que utilizaremos la librería `geopy` para convertir estos nombres a coordenadas.

In [2]:
import sys

# Instalar geopy si no está disponible
# La opción '--quiet' suprime la salida detallada de pip
!{sys.executable} -m pip install --quiet geopy

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pandas as pd

# Asegurarse de que el DataFrame 'df' esté disponible
if 'df' not in locals():
    raise NameError("DataFrame 'df' not found. Please ensure it's loaded.")

# Inicializar el geocodificador Nominatim
geolocator = Nominatim(user_agent="my-colab-app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# Obtener los países únicos de la columna 'pais-estreno'
unique_countries = df['pais-estreno'].dropna().unique()

# Geocodificar cada país único y almacenar sus coordenadas
country_coords = {}
for country in unique_countries:
    if pd.isna(country) or country == 'NI': # Handle missing values and 'NI' (Not Identified)
        continue
    try:
        location = geocode(country)
        if location:
            country_coords[country] = {'latitude': location.latitude, 'longitude': location.longitude}
        else:
            # Si no se encuentra la ubicación, se registran como None
            country_coords[country] = {'latitude': None, 'longitude': None}
    except Exception as e:
        print(f"Error al geocodificar {country}: {e}")
        country_coords[country] = {'latitude': None, 'longitude': None}

# Crear un DataFrame con las coordenadas geocodificadas
coords_df = pd.DataFrame.from_dict(country_coords, orient='index')
coords_df = coords_df.reset_index().rename(columns={'index': 'pais-estreno'})

# Unir las coordenadas al DataFrame original
df_with_coords = pd.merge(df, coords_df, on='pais-estreno', how='left')

# Eliminar filas donde no se pudieron encontrar coordenadas para el país de estreno
df_with_coords_filtered = df_with_coords.dropna(subset=['latitude', 'longitude'])

print(f"Países geocodificados: {len(country_coords)} de {len(unique_countries)}")
print(f"Filas con coordenadas válidas: {len(df_with_coords_filtered)}")

# Mostrar las primeras filas del DataFrame con las nuevas columnas de latitud y longitud
display(df_with_coords_filtered.head())

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 565, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/http/client.py", line 1430, in getresponse
    response.begin()
  File "/usr/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/http/client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 720, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/ssl.py", line

Países geocodificados: 23 de 24
Filas con coordenadas válidas: 222


,nombre-pelicula,anio,tipo-largometraje,duracion-min,tipo-filme,color,director-01,idioma,consol-idioma,lugar-prod,consol-lugar-prod,pais-rodaje,consol-pais-rodaje,,lugar-estreno,pais-estreno,sinopsis,latitude,longitude
0,La industria del salitre,1915,Largometraje documental,NI,35 mm,"Blanco & negro, Muda",Francisco Caamaño,Español,Español,Chile,Chile,Chile,Chile,"Teatros Setiembre y Politeama,Chile",Teatros Setiembre y Politeama,Chile,Largometraje sobre el proceso de extracción y ...,-31.761336,-71.31877
1,La industria del salitre,1915,Largometraje documental,NI,35 mm,"Blanco & negro, Muda",Francisco Caamaño,Español,Español,Chile,Chile,Chile,Chile,"Teatros Setiembre y Politeama,Chile",Teatros Setiembre y Politeama,Chile,Largometraje sobre el proceso de extracción y ...,-31.761336,-71.31877
2,La baraja de la muerte (o El enigma de la call...,1916,Largometraje de ficción,100,35 mm,"Blanco & negro, Muda",Salvador Giambastiani,Español,Español,Chile,Chile,Chile,Chile,"Teatros Colón y Novedades,Chile",Teatros Colón y Novedades,Chile,Basa su acción en un crimen cometido por Corin...,-31.761336,-71.31877
5,Alma chilena,1917,Largometraje de ficción,60,35 mm,"Blanco & negro, Muda",Arturo Mario,Español,Español,Chile,Chile,Chile,Chile,"Teatros Victoria y Colón,Chile",Teatros Victoria y Colón,Chile,Versión nacional de la argentina Nobleza Gauch...,-31.761336,-71.31877
6,Todo por la patria (o El girón de la bandera),1918,Largometraje de ficción,NI,35 mm,"Blanco & negro, Muda",Arturo Mario,Español,Español,Chile,Chile,Chile,Chile,"Teatros Colón y Alhambra,Chile",Teatros Colón y Alhambra,Chile,"Basada en la novela ""El girón de la bandera"". ...",-31.761336,-71.31877


In [9]:
import altair as alt
import pandas as pd
import sys

# Instalar vega_datasets si no está disponible
!{sys.executable} -m pip install --quiet vega_datasets
from vega_datasets import data

# Asegurarse de que df_with_coords_filtered esté disponible
if 'df_with_coords_filtered' not in locals():
    raise NameError("DataFrame 'df_with_coords_filtered' not found. Please run the geocoding cell first.")

# Data generators for the background (your original code)
sphere = alt.sphere()
graticule = alt.graticule()

# Source of land data
source = alt.topo_feature(data.world_110m.url, 'countries')

# Capa base del mapa (tu código original)
base_map = alt.layer(
    alt.Chart(sphere).mark_geoshape(fill='#ffffff'), # Color del océano
    alt.Chart(graticule).mark_geoshape(stroke='white', strokeWidth=0.5),
    alt.Chart(source).mark_geoshape(fill='#E1DFD6', stroke='black') # Color de la tierra
).project(
    'naturalEarth1'
).properties(width=800, height=500).configure_view(stroke=None)

# Capa de puntos para los países de estreno
# Ajustar el tamaño del punto según la cantidad de estrenos por país
# Para evitar superposición, agrupamos y contamos por país-estreno y luego unimos las coordenadas

premiere_counts = df_with_coords_filtered.groupby(['pais-estreno', 'latitude', 'longitude']).size().reset_index(name='num_premieres')

points_layer = alt.Chart(premiere_counts).mark_circle().encode(
    longitude='longitude:Q',
    latitude='latitude:Q',
    size=alt.Size('num_premieres:Q', title='Número de Estrenos', scale=alt.Scale(rangeMin=10, rangeMax=1000)), # Ajusta el rango para el tamaño de los puntos
    color=alt.value('#5A0F05'), # Color de los puntos de estreno
    tooltip=['pais-estreno', 'num_premieres']
).properties(
    title='El cine chileno alrededor del mundo'
)

# Combinar el mapa base y la capa de puntos
final_chart = base_map + points_layer

# Mostrar el gráfico
final_chart

alt.LayerChart(...)

In [10]:
# Exportar el gráfico a HTML
final_chart.save('paises_estreno_peliculas.html')